In [1]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction, MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol
from functools import partial

import qnexus as qnx
from copy import deepcopy
import matplotlib.pyplot as plt

# qibo's
import qibo
from qibo import gates, hamiltonians, models
from qibo.backends import GlobalBackend
from qibo.models.dbi.double_bracket import (
    DoubleBracketGeneratorType,
    DoubleBracketIteration,
)


# boostvqe's
from boostvqe import ansatze
from boostvqe.plotscripts import plot_gradients, plot_loss
from boostvqe.training_utils import Model, vqe_loss
from boostvqe.utils import (
    DBI_D_MATRIX,
    DBI_ENERGIES,
    DBI_FLUCTUATIONS,
    DBI_STEPS,
    FLUCTUATION_FILE,
    GRADS_FILE,
    HAMILTONIAN_FILE,
    LOSS_FILE,
    SEED,
    TOL,
    apply_dbi_steps,
    create_folder,
    generate_path,
    results_dump,
    rotate_h_with_vqe,
    train_vqe,
)
import numpy as np

/Users/pethidine/Library/Caches/pypoetry/virtualenvs/boostvqe-9Q-2T4PW-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# basics
nqubits = 5
h = 3

In [3]:
# helper functions
def exact_expectation_boost(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())

In [4]:
qibo.set_backend("tensorflow")

[Qibo 0.2.12|INFO|2024-10-14 15:26:12]: Using tensorflow backend on /device:CPU:0
INFO:qibo.config:Using tensorflow backend on /device:CPU:0


In [5]:
# build hamiltonian
ham_boost = hamiltonians.TFIM(nqubits=nqubits, h=h, dense=False)
print(ham_boost.matrix)

[Qibo 0.2.12|WARNING|2024-10-14 15:26:17]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


tf.Tensor(
[[-5.+0.j -3.+0.j -3.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-3.+0.j -1.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-3.+0.j  0.+0.j -1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j  0.+0.j -3.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -1.+0.j -3.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ... -3.+0.j -3.+0.j -5.+0.j]], shape=(32, 32), dtype=complex128)


In [6]:
# build ansatz circuit
nlayer = 1
ansatz_circ = ansatze.hdw_efficient(nqubits, nlayer)
print(ansatz_circ.draw())

[Qibo 0.2.12|WARNING|2024-10-14 15:26:23]: Starting on qibo 0.2.13, ``Circuit.draw`` will work in-place. The in-place method is currently implemented as ``Circuit.display``, but will be renamed as ``Circuit.draw`` on release 0.2.13. In release 0.2.12, the in-place display of circuits is accessible as ``Circuit.display``.


q0: ─RY─RZ─o───RY─RZ───o─RY─
q1: ─RY─RZ─Z───RY─RZ─o─|─RY─
q2: ─RY─RZ───o─RY─RZ─Z─|─RY─
q3: ─RY─RZ───Z─RY─RZ───|─RY─
q4: ─RY─RZ─────RY─RZ───Z─RY─


In [8]:
# build zero state
zero_state = ham_boost.backend.zero_state(nqubits)
# initial params
params_len = len(ansatz_circ.get_parameters())
# fix numpy seed to ensure replicability of the experiment
seed = 10
np.random.seed(seed)
initial_params = np.random.uniform(-np.pi, np.pi, params_len)
print(initial_params)
# initial energy
c0 = deepcopy(ansatz_circ)
c0.set_parameters(initial_params)
target_energy = np.real(np.min(np.asarray(ham_boost.eigenvalues())))
print('Target enegry:', target_energy)
print('Initial energy:', exact_expectation_boost(ham_boost, c0))

[ 1.70475788 -3.01120431  0.83973663  1.5632809  -0.00938072 -1.72915367
 -1.89712697  1.63696274 -2.07903793 -2.58653723  1.16465009  2.84875441
 -3.11678496  0.07660625  1.96425543  0.70702213  1.39332975 -1.30768123
  2.62495223  1.34821941  0.26731415 -2.2483119  -0.79582348  1.09411377
 -0.36547294]
Target enegry: -15.422871679540702
Initial energy: tf.Tensor(-2.651967569509104, shape=(), dtype=float64)


In [11]:
niter = 3
# define the qibo loss function
objective_boost = partial(vqe_loss)
# logging hisotry
params_history, loss_history, grads_history, fluctuations = [], [], [], []
# set optimizer
optimizer = 'cma'
tol = 1e-2

In [12]:
# train vqe
(
    partial_results,
    partial_params_history,
    partial_loss_history,
    partial_grads_history,
    partial_fluctuations,
    vqe,
) = train_vqe(
    deepcopy(ansatz_circ),
    ham_boost,  # Fixed hamiltonian
    optimizer,
    initial_params,
    tol=tol,
    niterations=1,
    nmessage=1,
    loss=objective_boost,
)
params_history = np.array(partial_params_history)
loss_history = np.array(partial_loss_history)
grads_history = np.array(partial_grads_history)
fluctuations = np.array(partial_fluctuations)

INFO:root:Optimization iteration 0/1
INFO:root:Loss -2.652
INFO:root:Minimize the energy


(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=1017422, Mon Oct 14 15:28:45 2024)


INFO:root:Optimization iteration 1/1
INFO:root:Loss 1.4378
INFO:root:Optimization iteration 2/1
INFO:root:Loss 5.626
INFO:root:Optimization iteration 3/1
INFO:root:Loss -2.774
INFO:root:Optimization iteration 4/1
INFO:root:Loss 7.7409
INFO:root:Optimization iteration 5/1
INFO:root:Loss 3.1651
INFO:root:Optimization iteration 6/1
INFO:root:Loss 3.2266
INFO:root:Optimization iteration 7/1
INFO:root:Loss -0.95215
INFO:root:Optimization iteration 8/1
INFO:root:Loss -7.6397
INFO:root:Optimization iteration 9/1
INFO:root:Loss -2.7641
INFO:root:Optimization iteration 10/1
INFO:root:Loss 0.60643
INFO:root:Optimization iteration 11/1
INFO:root:Loss -0.049027
INFO:root:Optimization iteration 12/1
INFO:root:Loss 5.7536
INFO:root:Optimization iteration 13/1
INFO:root:Loss 1.1794
INFO:root:Optimization iteration 14/1
INFO:root:Loss 0.5403
INFO:root:Optimization iteration 15/1
INFO:root:Loss 0.010682
INFO:root:Optimization iteration 16/1
INFO:root:Loss 3.82
INFO:root:Optimization iteration 17/1
INFO

KeyboardInterrupt: 